In [101]:
# This is an implementation of imp step based on Gorte paper 

In [145]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from FIS1 import FIS1


In [146]:
def get_bearing(point1, point2):
    # this code calculate the bearing of any given pair of longitude, latitude  
    geodesic = pyproj.Geod(ellps='WGS84')
    fwd_azimuth,back_azimuth,distance = geodesic.inv(point1[0], point1[1], point2[0], point2[1])
    return fwd_azimuth

def edge_bearing(edge):
    # this function calculate the bearing from the starting and ending node of each road segment
    bearing = get_bearing(edge[0], edge[len(edge) - 1])
    return(bearing)
    
def conv_angle(angle):
    # this function convert angle from -pi,pi to 0,2*pi
    if angle < 0 :
        angle = angle + 360
    return(angle)

def conc(a):
    #function to convert list or integer in osmid into a unique string id 
    if type(a) is int:
        return str(a)
    ans = ",".join(map(str, a))
    return ans

def err_polygon(curr_loc, err_size):
    # function that output shapely polygon for point error bound
    x = curr_loc['geometry'].iloc[0].x
    y = curr_loc['geometry'].iloc[0].y
    
    err_coord = [[x - err_size, y + err_size], 
                 [x + err_size, y + err_size],
                 [x + err_size, y - err_size],
                 [x - err_size, y - err_size]]

    poly_coord = Polygon(err_coord)
    # #print(ply_coord)
    df = {'Attribute' : ['name1'], 'geometry':poly_coord}

    #projected to UTM 31 
    err_poly = gpd.GeoDataFrame(df, geometry = 'geometry', crs = "EPSG:32631")
    
    return err_poly

# Project data into UTM 31 

In [147]:
gdf_utm = pd.read_pickle('gdf_utm.pkl')
edges_utm = pd.read_pickle('edges_utm.pkl')
edges_utm

osmid   lanes  ref  \
u           v          key                                                   
21518865    43601038   0    [692976089, 692976090, 801727033]       3  K 8   
21518866    21518865   0      [714428680, 4064467, 820160405]  [3, 2]  NaN   
21518867    21518866   0      [666953657, 4064467, 714428681]       2  NaN   
            60358392   0    [1021975576, 666953658, 30374499]       3  NaN   
21518872    52774658   0                [30374498, 666953655]  [3, 2]  NaN   
...                                                       ...     ...  ...   
6715626972  6715626969 0                            714435226       1  NaN   
9776435595  1932959986 0                             17438340     NaN  NaN   
            6715626972 0                             17438340     NaN  NaN   
            6715626969 0                           1064505851     NaN  NaN   
10548918654 6152925224 0                             28961154     NaN  NaN   

                                                            name  \
u           v          key                                         
21518865    43601038   0                             Hafenstraße   
21518866    21518865   0                             Engelstraße   
21518867    21518866   0                             Engelstraße   
            60358392   0                          Herwarthstraße   
21518872    52774658   0    [Von-Steuben-Straße, Berliner Platz]   
...                                                          ...   
6715626972  6715626969 0                                     NaN   
9776435595  1932959986 0                          Berliner Platz   
            6715626972 0                          Berliner Platz   
            6715626969 0                                     NaN   
10548918654 6152925224 0                             Brunostraße   

                                 highway maxspeed  oneway  reversed   length  \
u           v          key                                                     
21518865    43601038   0       secondary       50   False     False   29.023   
21518866    21518865   0     residential       30    True     False   99.176   
21518867    21518866   0     residential       30    True     False  124.335   
            60358392   0       secondary       30    True     False   56.497   
21518872    52774658   0       secondary       30    True     False   64.592   
...                                  ...      ...     ...       ...      ...   
6715626972  6715626969 0    unclassified      NaN    True     False   59.092   
9776435595  1932959986 0    unclassified      NaN   False     False   27.814   
            6715626972 0    unclassified      NaN   False      True    4.837   
            6715626969 0    unclassified      NaN    True     False   57.953   
10548918654 6152925224 0     residential       30   False     False   97.885   

                                                                     geometry  \
u           v          key                                                      
21518865    43601038   0    LINESTRING (818076.354 5766103.217, 818080.537...   
21518866    21518865   0    LINESTRING (818118.595 5766193.172, 818108.090...   
21518867    21518866   0    LINESTRING (818166.493 5766308.115, 818164.278...   
            60358392   0    LINESTRING (818166.493 5766308.115, 818171.377...   
21518872    52774658   0    LINESTRING (818280.573 5766262.212, 818286.118...   
...                                                                       ...   
6715626972  6715626969 0    LINESTRING (818368.292 5766296.311, 818372.745...   
9776435595  1932959986 0    LINESTRING (818373.118 5766295.771, 818376.985...   
            6715626972 0    LINESTRING (818373.118 5766295.771, 818369.688...   
            6715626969 0    LINESTRING (818373.118 5766295.771, 818377.187...   
10548918654 6152925224 0    LINESTRING (819545.555 5766343.170, 819547.189...   

                           width access  \
u           v     

# Project trajectory into utm 31

# Perform IMP 

In [148]:
# initialization for IMP
count = 0 
same_link = 0
stop_iter = False
iter = 0
err_size = 38

# saving answer for debugging purposes 
# edge_link saves all the candidate link name for each iteration 
# final answer is stored in the edge_link variable 
edge_link = []
# fis_res saves the output of FIS algorithm at every iteration 
fis_res = []
# HE_iter saves the heading error values for each candidate edge at any given iteration 
HE_iter = [] 
#curr_pos
curr_pos_list = []
# save candidate link name each iteration  
candidate_link_res = []


while stop_iter == False :
    # extract current location at given iteration 
    curr_loc = gdf_utm.iloc[iter].to_frame().T
    # save the iteration current position as a list
    curr_pos_list.append(curr_loc)
    
    #-----------------------------------------------------------------------
    # input should be location and error size 
    # create rectangular polygon 
    err_poly = err_polygon(curr_loc, err_size)

    # to plot error polygon for debugging
    # err_poly.plot()

    #---------------------------------------------------------------------------
    
    # Check for intersection and containment using geopandas
    intersects = gpd.sjoin(err_poly, edges_utm, op='intersects')
    contains = gpd.sjoin(err_poly, edges_utm, op='contains')

    if (len(intersects) + len(contains)) <=0:
        print(['no edeges intersects with error bound at iteration number', iter + 1])
    else:    
        # perform IMP only when there is edge intersects with error bound
        print(['edges found at iteration number', iter + 1])

        # extract index from edges that intersect with error polygon 
        int_index = intersects[['index_right0', 'index_right1', 'index_right2']]
        # extract index from edges that contained in the error polygon 
        cont_index = contains[['index_right0', 'index_right1', 'index_right2']]

        # merge index
        index = pd.concat([int_index, cont_index])
        # drop duplicate
        index = index.drop_duplicates()

        # initialize candidate edges 
        appended_edge = []

        # extract candidate eges  
        for i in range(len(index)):
            edge_list = (index['index_right0'].iloc[i], index['index_right1'].iloc[i], 0 )
            appended_edge.append(edge_list)

        candidate_link = edges_utm.loc[appended_edge]

        #save candidate link name 
        candidate_link_res.append(candidate_link['osmid'])

        # calculate perpendicular distance 
        # initialize list that hold perpendicular distance between points and edges
        p_dist = []

        # calculate perpendicular distance between current point and 
        for i in range(len(candidate_link)):
            p_dist.append(candidate_link['geometry'].iloc[i].distance(curr_loc['geometry']).iloc[0])

        # attach perpendicular distance to candidate link 
        candidate_link["perp_dist"] = p_dist

        # print(candidate_link)

        # calculate heading error
        # convert lat lon into tupple coordinate 
        candidate_link['lon_lat_pair'] = candidate_link.lon_lat.apply(lambda geom: list(geom.coords))

        # calculate bearing frome start and end node for each candidate link (see notes below)
        bearing_raw = candidate_link['lon_lat_pair'].apply(edge_bearing)

        # convert bearing from -pi, pi to 0, 2pi range
        candidate_link['edge_heading'] = bearing_raw.apply(conv_angle)

        # heading error = abs(gps heading - edge bearing)
        candidate_link['heading_error'] = abs(candidate_link['edge_heading'] - gdf_utm['GPS Bearing'].iloc[iter])

        # initialize input for FIS
        PD = candidate_link['perp_dist']
        HE = candidate_link['heading_error']
        speed = np.repeat(gdf_utm['speed_mps'][iter]/ 3.6, len(candidate_link))
        hdop = np.repeat(gdf_utm['GPS HDOP'][iter], len(candidate_link))

        # save HE value every iter 
        HE_iter.append(HE)
        # rearrange new data to the input of fis1  
        new_data = np.array([speed, HE, PD, hdop]).T

        # calculating FIS
        pred =[]
        for i in range(len(new_data)):
            pred.append(FIS1(new_data[i,:], plot = False))

        # print(pred)
        # save fis result 
        fis_res.append(pred)

        # pick candidate link based on 
        index = pred.index(max(pred))

        edge_link.append(candidate_link['osmid'].iloc[index])

        # check if the current position and previous position is in the same edge
        if count > 0:
            if edge_link[count] == edge_link[count - 1]:
                same_link = same_link + 1
            else:
                same_link = 0

        # check to stop the for loop if three points belong to the same edge
        if same_link == 1:
            print(['stopping at ',iter])
            stop_iter = True
            # find projection point 
            matched_link = candidate_link.iloc[index]
            dist = matched_link['geometry'].project(curr_loc['geometry']).iloc[0]
            matched_point = list(matched_link['geometry'].interpolate(dist).coords)
            matched_point = gpd.GeoDataFrame(geometry=gpd.points_from_xy([matched_point[0][0]], [matched_point[0][1]]), crs="EPSG:32631")
        else:
            count = count + 1

    #update iteration 
    iter = iter + 1     

C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Pl

['no edeges intersects with error bound at iteration number', 1]
['no edeges intersects with error bound at iteration number', 2]
['no edeges intersects with error bound at iteration number', 3]
['no edeges intersects with error bound at iteration number', 4]
['no edeges intersects with error bound at iteration number', 5]
['no edeges intersects with error bound at iteration number', 6]


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Pl

['no edeges intersects with error bound at iteration number', 7]
['no edeges intersects with error bound at iteration number', 8]
['no edeges intersects with error bound at iteration number', 9]
['no edeges intersects with error bound at iteration number', 10]
['no edeges intersects with error bound at iteration number', 11]
['edges found at iteration number', 12]
['edges found at iteration number', 13]
['stopping at ', 12]


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: Th

In [149]:
x = candidate_link['lon_lat_pair'].iloc[0][0]
y = candidate_link['lon_lat_pair'].iloc[0][1]

get_bearing(x, y)
x

(7.651807, 51.9548654)

# Plot Result

In [153]:
%matplotlib tk
# This is how we  visualize edges and error bound 

# find which edges is selected at time point
# find index of the edge id
loc = np.where(edges_utm["str_id"] == conc(edge_link[count]))

# find the last two position for IMP
poly_1 = err_polygon(curr_pos_list[count + iter - 2], err_size)
poly_2 = err_polygon(curr_pos_list[count + iter - 3], err_size)

#Save selected edge 
answer_loc = edges_utm.iloc[loc]

# plotting edges and starting point together 
f, ax = plt.subplots()

# location for all point
#locs_utm.plot(ax=ax)
point_locs = gdf_utm['geometry'].to_frame()
point_locs.iloc[0:(iter), :].plot(ax = ax)

#err coord 
# better if we just take location at the last and use error bound function 
poly_1.plot(ax=ax, facecolor="none")
poly_2.plot(ax=ax, facecolor="none")

# this plot all the road system 
edges_utm.plot(ax=ax)

# this plot the selected edge at time point 
answer_loc.plot(ax=ax, cmap = "Reds")

# matched point plot
matched_point.plot(ax = ax, color = "Green")

#print(intersects['index_right'])

<Axes: >

In [107]:
# to convert current position list from list to dataframe 
# pd.concat(curr_pos_list)['geometry']

0     POINT (819633.5807833129 5766179.609685688)
1     POINT (819622.5537502016 5766163.388639316)
2     POINT (819626.0013702163 5766163.385287559)
3     POINT (819632.6479374802 5766162.987711843)
4     POINT (819643.6418708554 5766158.897275649)
5     POINT (819649.3764079262 5766159.465666196)
6     POINT (819653.2070335334 5766153.690892602)
7      POINT (819656.3996371442 5766158.78538911)
8      POINT (819660.8047981295 5766167.51956023)
9     POINT (819661.9277071179 5766176.194010578)
10     POINT (819666.404511811 5766203.248064032)
11     POINT (819649.308062795 5766214.664886044)
12    POINT (819609.7343499986 5766228.984348884)
Name: geometry, dtype: object

In [144]:
# save saved edge location for SMP1 debugging purposes
answer_loc.to_pickle('current_edge.pkl')

# Debugging 

In [14]:
iter = 14
# convert ot UTM zone 31 which is equal to epsg32631 according to https://spatialreference.org/ref/epsg/32631/
curr_loc = gdf_utm.iloc[iter].to_frame().T
curr_pos_list.append(curr_loc)
#-----------------------------------------------------------------------
# this part could be made into a function that create error polygon 
# input should be location and error size 
# create rectangular polygon 
err_poly = err_polygon(curr_loc, err_size)

# plot error polygon for debugging
# err_poly.plot()

#---------------------------------------------------------------------------

# Check for intersection and containment using geopandas
intersects = gpd.sjoin(err_poly, edges_utm, op='intersects')
contains = gpd.sjoin(err_poly, edges_utm, op='contains')
# skip iteration if no edges are detected 
if (len(intersects) + len(contains)) >0:
    print(['edges found at iteration number', iter + 1])

    # extract index from edges that intersect with error polygon 
    int_index = intersects[['index_right0', 'index_right1', 'index_right2']]
    # extract index from edges that contained in the error polygon 
    cont_index = contains[['index_right0', 'index_right1', 'index_right2']]

    # merge index
    index = pd.concat([int_index, cont_index])
    # drop duplicate
    index = index.drop_duplicates()

    # initialize candidate edges 
    appended_edge = []

    # extract candidate eges  
    for i in range(len(index)):
        edge_list = (index['index_right0'].iloc[i], index['index_right1'].iloc[i], 0 )
        appended_edge.append(edge_list)

    candidate_link = edges_utm.loc[appended_edge]

    #save candidate link name 
    candidate_link_res.append(candidate_link['osmid'])

    # calculate perpendicular distance 
    # initialize list that hold perpendicular distance between points and edges
    p_dist = []

    # calculate perpendicular distance between current point and 
    for i in range(len(candidate_link)):
        p_dist.append(candidate_link['geometry'].iloc[i].distance(curr_loc['geometry']).iloc[0])

    # attach perpendicular distance to candidate link 
    candidate_link["perp_dist"] = p_dist

    # print(candidate_link)

    # calculate heading error
    # convert lat lon into tupple coordinate 
    candidate_link['lon_lat_pair'] = candidate_link.lon_lat.apply(lambda geom: list(geom.coords))

    # calculate bearing frome start and end node for each candidate link (see notes below)
    bearing_raw = candidate_link['lon_lat_pair'].apply(edge_bearing)

    # convert bearing from -pi, pi to 0, 2pi range
    candidate_link['edge_heading'] = bearing_raw.apply(conv_angle)

    # heading error = abs(gps heading - edge bearing)
    candidate_link['heading_error'] = abs(candidate_link['edge_heading'] - gdf_utm['GPS Bearing'].iloc[iter])

    # initialize input for FIS
    PD = candidate_link['perp_dist']
    HE = candidate_link['heading_error']
    speed = np.repeat(gdf_utm['GPS Speed'][iter], len(candidate_link))
    hdop = np.repeat(gdf_utm['GPS HDOP'][iter], len(candidate_link))

    # save HE value every iter 
    HE_iter.append(HE)
    # rearrange new data to the input of fis1  
    new_data = np.array([speed, HE, PD, hdop]).T

    # calculating FIS
    pred =[]
    for i in range(len(new_data)):
        pred.append(FIS1(new_data[i,:], plot = False))

    # print(pred)
    # save fis result 
    fis_res.append(pred)

    # pick candidate link based on 
    index = pred.index(max(pred))
    
    edge_link.append(matched_link['osmid'])

    # check if the current position and previous position is in the same edge
    if count > 0:
        if edge_link[count] == edge_link[count - 1]:
            same_link = same_link + 1
        else:
            same_link = 0

    # check to stop the for loop if three points belong to the same edge
    if same_link == 1:
        stop_iter = True
    else:
        count = count + 1

#update iteration 
iter = iter + 1 


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


['edges found at iteration number', 15]


C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\j_3r3\.conda\envs\ox\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Cov

In [127]:
candidate_link

osmid lanes    ref  \
u         v         key                                        
53200191  135535395 0    [673214248, 234046511]     3  L 793   
135535395 53200191  0    [673214248, 234046511]     3  L 793   
          135535401 0                  14180413     2    NaN   
135535401 135535395 0                  14180413     2    NaN   

                                         name      highway maxspeed  oneway  \
u         v         key                                                       
53200191  135535395 0        Wolbecker Straße    secondary       50   False   
135535395 53200191  0        Wolbecker Straße    secondary       50   False   
          135535401 0    Andreas-Hofer-Straße  residential      NaN   False   
135535401 135535395 0    Andreas-Hofer-Straße  residential      NaN   False   

                         reversed  length  \
u         v         key                     
53200191  135535395 0        True  54.889   
135535395 53200191  0       False  54.889   
          135535401 0       False  80.186   
135535401 135535395 0        True  80.186   

                                                                  geometry  \
u         v         key                                                      
53200191  135535395 0    LINESTRING (819585.705 5766245.100, 819590.522...   
135535395 53200191  0    LINESTRING (819640.807 5766244.782, 819631.704...   
          135535401 0    LINESTRING (819640.807 5766244.782, 819640.611...   
135535401 135535395 0    LINESTRING (819638.028 5766325.038, 819638.661...   

                        width access  \
u         v         key                
53200191  135535395 0     NaN    NaN   
135535395 53200191  0     NaN    NaN   
          135535401 0     NaN    NaN   
135535401 135535395 0     NaN    NaN   

                                                                   lat_lon  \
u         v         key                                                      
53200191  135535395 0    LINESTRING (7.65181 51.95487, 7.65188 51.95486...   
135535395 53200191  0    LINESTRING (7.65261 51.95483, 7.65247 51.95483...   
          135535401 0    LINESTRING (7.65261 51.95483, 7.65261 51.95488...   
135535401 135535395 0    LINESTRING (7.65264 51.95555, 7.65263 51.95533...   

                                      str_id  perp_dist  
u         v         key                                  
53200191  135535395 0    673214248,234046511  31.293527  
135535395 53200191  0    673214248,234046511  31.293527  
          135535401 0               14180413  31.293527  
135535401 135535395 0               14180413  31.293527